# This app focuses on extracting cryptocurrencies data from coingecko's API

## APIs used:

    - Get/coins/markets to get a list of all supported coins, ath, price, volume and other info
    - Get/coins/{id}/history to get community and developer data


In [1]:
import requests
import pandas as pd
import time

In [2]:
# Create an empty list to hold the coins information
crypto_list = []

In [3]:
# Find an API that has information on coin

# Make a loop to get 10 pages
for i in range(1,10,1):
    page_number = i
    base_url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page={page_number}&sparkline=false"
    # Test out the URL
    #print(base_url)

    # Request json file from the url
    response_json = requests.get(base_url).json()

    # append response to empty list
    crypto_list.append(response_json)

In [4]:
# Add all the data frames together
# Set up the a value for the previous dataframe
previous_df = pd.DataFrame(crypto_list[0])
for i in range(1, len(crypto_list)):
    next_df = pd.DataFrame(crypto_list[i])
    complete_df = pd.concat([previous_df, next_df])
    previous_df = complete_df



In [5]:
clean_complete_df = complete_df.reset_index(drop=True)
#clean_complete_df

In [6]:
# See the name of the columns
#clean_complete_df.columns

In [7]:
# Only get specific columns
crypto_df = clean_complete_df[['id', 'symbol', 'current_price','market_cap', 'market_cap_rank', 'ath', 'ath_change_percentage', 'ath_date','atl', 'atl_change_percentage', 'atl_date']]

In [8]:
# Only run this cell if the data hasn't been renamed and cleaned
if crypto_df.columns[3] == "market_cap":
    # Clean data by dropping columns, ONLY RUN IT ONCE, since columns will change name
    #crypto_df = crypto_df[['id', 'symbol', 'current_price','market_cap', 'market_cap_rank', 'ath', 'ath_change_percentage', 'ath_date','atl', 'atl_change_percentage', 'atl_date']]
    crypto_df['market_cap'] = crypto_df['market_cap']/1000000000

    # Rename columns to see easier
    crypto_df.rename(columns={'market_cap':'market_cap_billion(2021)',
                              'current_price' : 'price_04-22-21'

                             }, inplace=True)


<ipython-input-8-3d38269c4a46>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crypto_df['market_cap'] = crypto_df['market_cap']/1000000000
/Users/johannvillalvir/opt/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [9]:
# Verify data drame
#crypto_df.head()

In [10]:
# Save data as csv
crypto_df.to_csv("data/cryptocurrencies_data_2021.csv")

In [11]:
# Get each coin's price on 01-04-2018 to now
# Make empty list to hold the value of coins

coin_data_2018 = []

target_date = "01-04-2018"

In [12]:
# Set up timer to slow down requests speed
def timer_milliseconds(milliseconds = 3):
    for i in range(0, milliseconds):
        print(f"Counting... {i +1}")
        time.sleep(1)
        

In [13]:
# Make function to perform request based on specific data, use dataframe above's id to search that specific coin
def obtain_metrics_by_date (target_df = crypto_df, target_list = coin_data_2018, date='01-04-2018'):
    # Set up error counter to break out if there are too many errors
    key_error_counter = 0
    timeout_error_counter = 0
    #iterate through dataframe gettind index and row
    for index, row in target_df.iterrows():
        
        #get coin name through dictionary syntax
        target_coin = row['id']
        
        # print the current row's coin
        print(f"Processing {target_coin}")
        
        # number of key_errors allowed 
        if (key_error_counter < 150) and (timeout_error_counter < 2):
            try:
                # build url
                price_url = f"https://api.coingecko.com/api/v3/coins/{target_coin}/history?date={date}localization=False"
                print(f"URL: {price_url}")
                print(f"At index: {index}")
                # save json respons
                response_json = requests.get(price_url).json()
                # traverse json getting specific data
                coin_id = response_json['id']
                coin_price = response_json['market_data']['current_price']['usd']
                coin_reddit_subscribers = response_json['community_data']['reddit_subscribers']
                coin_twitter_followers = response_json['community_data']['twitter_followers']
                coin_alexa_rank = response_json['public_interest_stats']['alexa_rank']
                # Add data to empty list in dictionary style
                target_list.append({
                    "id" : coin_id,
                    f"price {date}" : coin_price,
                    f"reddit_subscribers {date}" : coin_reddit_subscribers,
                    f"twitter_followers {date}" : coin_twitter_followers,
                    f"alexa_rank {date}" : coin_alexa_rank
                       })
            # The first error is a no market_data for that coin
            except :
                try:
                    # The first error will be a key error, since young coins dont have 'market_data'
                    # add one to the key_error counter
                    key_error_counter+=1
                    #print message
                    print("Key error. Coin too young to find all data")   
                    print(f"{target_coin}")
                    print(price_url)
                    # make a request again, in case we got timedout
                    response_json = requests.get(price_url).json()
                    #still add the coin to the list
                    target_list.append({'id': coin_id})
                 
                # the next error would be a timedout error
                except :
                    print('Ruh Roh! Looks like you have been timed out')
                    print(f"The last coin to be pulled was {target_coin} at index {index}")
                    # If we got timed out, we want to exit the function
                    timeout_error_counter += 1
                    timer_milliseconds(30)
                    #break
        # print message for too many requests errors
        else:
            print(f"You have reached {key_error_counter} key errors and {timeout_error_counter} timeouts.")
            print(f"Last data pulled at position index:{index}, id: {target_coin}")
            break
    print("Done with Data Retieval")
    print(f"We found a total of {len(target_list)} entries")
  



In [14]:
obtain_metrics_by_date()

Processing bitcoin
URL: https://api.coingecko.com/api/v3/coins/bitcoin/history?date=01-04-2018localization=False
At index: 0
Processing ethereum
URL: https://api.coingecko.com/api/v3/coins/ethereum/history?date=01-04-2018localization=False
At index: 1
Processing binancecoin
URL: https://api.coingecko.com/api/v3/coins/binancecoin/history?date=01-04-2018localization=False
At index: 2
Processing ripple
URL: https://api.coingecko.com/api/v3/coins/ripple/history?date=01-04-2018localization=False
At index: 3
Processing tether
URL: https://api.coingecko.com/api/v3/coins/tether/history?date=01-04-2018localization=False
At index: 4
Processing cardano
URL: https://api.coingecko.com/api/v3/coins/cardano/history?date=01-04-2018localization=False
At index: 5
Processing dogecoin
URL: https://api.coingecko.com/api/v3/coins/dogecoin/history?date=01-04-2018localization=False
At index: 6
Processing polkadot
URL: https://api.coingecko.com/api/v3/coins/polkadot/history?date=01-04-2018localization=False
At

Key error. Coin too young to find all data
algorand
https://api.coingecko.com/api/v3/coins/algorand/history?date=01-04-2018localization=False
Processing maker
URL: https://api.coingecko.com/api/v3/coins/maker/history?date=01-04-2018localization=False
At index: 40
Processing kusama
URL: https://api.coingecko.com/api/v3/coins/kusama/history?date=01-04-2018localization=False
At index: 41
Key error. Coin too young to find all data
kusama
https://api.coingecko.com/api/v3/coins/kusama/history?date=01-04-2018localization=False
Processing dai
URL: https://api.coingecko.com/api/v3/coins/dai/history?date=01-04-2018localization=False
At index: 42
Key error. Coin too young to find all data
dai
https://api.coingecko.com/api/v3/coins/dai/history?date=01-04-2018localization=False
Processing avalanche-2
URL: https://api.coingecko.com/api/v3/coins/avalanche-2/history?date=01-04-2018localization=False
At index: 43
Key error. Coin too young to find all data
avalanche-2
https://api.coingecko.com/api/v3/co

In [15]:
len(coin_data_2018)

54

In [16]:
# If we got timed out, make a DF starting from the last token proccesed to run the function again
last_coin_proccesed = len(coin_data_2018)
left_over_df = crypto_df.iloc[last_coin_proccesed:]
left_over_df



,id,symbol,price_04-22-21,market_cap_billion(2021),market_cap_rank,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date
54,holotoken,hot,0.013749,2.442933,55,0.031267,-56.02784,2021-04-05T16:04:52.608Z,0.000220,6143.76333,2020-03-13T02:23:01.612Z
55,chiliz,chz,0.458628,2.440091,56,0.878633,-47.80213,2021-03-13T08:04:21.200Z,0.004109,11061.88542,2019-09-28T00:00:00.000Z
56,havven,snx,15.990000,2.433155,57,28.530000,-43.95377,2021-02-14T01:12:38.505Z,0.034786,45869.89275,2019-01-06T00:00:00.000Z
57,celsius-degree-token,cel,6.170000,2.365716,58,7.960000,-22.48595,2021-04-09T10:19:25.352Z,0.026203,23441.49470,2018-12-07T00:00:00.000Z
58,enjincoin,enj,2.480000,2.323566,59,3.940000,-36.92484,2021-04-09T12:29:27.777Z,0.018660,13227.81602,2017-11-12T00:00:00.000Z
...,...,...,...,...,...,...,...,...,...,...,...
2245,bitcoin-incognito,xbi,0.009503,0.000156,2245,0.695694,-98.63406,2018-11-02T03:03:20.015Z,0.001440,559.84334,2021-03-28T00:24:55.615Z
2246,2give,2give,0.000290,0.000153,2246,0.049980,-99.41932,2017-12-25T00:00:00.000Z,0.000007,3849.57681,2019-11-05T08:12:39.391Z
2247,evil-coin,evil,0.007166,0.000153,2247,0.068920,-89.60175,2018-07-06T03:44:58.098Z,0.000574,1148.80075,2019-10-08T00:00:00.000Z
2248,advanced-internet-block,aib,0.015678,0.000151,2248,0.182530,-91.41100,2018-01-16T00:00:00.000Z,0.000121,12869.79951,2020-11-21T10:39:58.741Z


In [17]:
# Keep obtaining data until our df is at least 500 short from the original list
while len(coin_data_2018) < len(crypto_df) -500:
    obtain_metrics_by_date(left_over_df, coin_data_2018)
    # Get the index for the last coin pricesses
    last_coin_proccesed = len(coin_data_2018)
    # Update the leftovers data frame starting from the last coin_processed
    left_over_df = crypto_df.iloc[last_coin_proccesed:]



Processing holotoken
URL: https://api.coingecko.com/api/v3/coins/holotoken/history?date=01-04-2018localization=False
At index: 54
Key error. Coin too young to find all data
holotoken
https://api.coingecko.com/api/v3/coins/holotoken/history?date=01-04-2018localization=False
Ruh Roh! Looks like you have been timed out
The last coin to be pulled was holotoken at index 54
Counting... 1
Counting... 2
Counting... 3
Counting... 4
Counting... 5
Counting... 6
Counting... 7
Counting... 8
Counting... 9
Counting... 10
Counting... 11
Counting... 12
Counting... 13
Counting... 14
Counting... 15
Counting... 16
Counting... 17
Counting... 18
Counting... 19
Counting... 20
Counting... 21
Counting... 22
Counting... 23
Counting... 24
Counting... 25
Counting... 26
Counting... 27
Counting... 28
Counting... 29
Counting... 30
Processing chiliz
URL: https://api.coingecko.com/api/v3/coins/chiliz/history?date=01-04-2018localization=False
At index: 55
Key error. Coin too young to find all data
chiliz
https://api.co

Processing wazirx
URL: https://api.coingecko.com/api/v3/coins/wazirx/history?date=01-04-2018localization=False
At index: 90
Key error. Coin too young to find all data
wazirx
https://api.coingecko.com/api/v3/coins/wazirx/history?date=01-04-2018localization=False
Processing bancor
URL: https://api.coingecko.com/api/v3/coins/bancor/history?date=01-04-2018localization=False
At index: 91
Processing paxos-standard
URL: https://api.coingecko.com/api/v3/coins/paxos-standard/history?date=01-04-2018localization=False
At index: 92
Key error. Coin too young to find all data
paxos-standard
https://api.coingecko.com/api/v3/coins/paxos-standard/history?date=01-04-2018localization=False
Processing xdce-crowd-sale
URL: https://api.coingecko.com/api/v3/coins/xdce-crowd-sale/history?date=01-04-2018localization=False
At index: 93
Key error. Coin too young to find all data
xdce-crowd-sale
https://api.coingecko.com/api/v3/coins/xdce-crowd-sale/history?date=01-04-2018localization=False
Processing reserve-rig

Key error. Coin too young to find all data
venus
https://api.coingecko.com/api/v3/coins/venus/history?date=01-04-2018localization=False
Processing bmax
URL: https://api.coingecko.com/api/v3/coins/bmax/history?date=01-04-2018localization=False
At index: 116
Key error. Coin too young to find all data
bmax
https://api.coingecko.com/api/v3/coins/bmax/history?date=01-04-2018localization=False
Processing livepeer
URL: https://api.coingecko.com/api/v3/coins/livepeer/history?date=01-04-2018localization=False
At index: 117
Key error. Coin too young to find all data
livepeer
https://api.coingecko.com/api/v3/coins/livepeer/history?date=01-04-2018localization=False
Processing nxm
URL: https://api.coingecko.com/api/v3/coins/nxm/history?date=01-04-2018localization=False
At index: 118
Key error. Coin too young to find all data
nxm
https://api.coingecko.com/api/v3/coins/nxm/history?date=01-04-2018localization=False
Processing lisk
URL: https://api.coingecko.com/api/v3/coins/lisk/history?date=01-04-201

Key error. Coin too young to find all data
wootrade-network
https://api.coingecko.com/api/v3/coins/wootrade-network/history?date=01-04-2018localization=False
Processing bitcoin-diamond
URL: https://api.coingecko.com/api/v3/coins/bitcoin-diamond/history?date=01-04-2018localization=False
At index: 147
Processing maidsafecoin
URL: https://api.coingecko.com/api/v3/coins/maidsafecoin/history?date=01-04-2018localization=False
At index: 148
Processing compound-uniswap
URL: https://api.coingecko.com/api/v3/coins/compound-uniswap/history?date=01-04-2018localization=False
At index: 149
Key error. Coin too young to find all data
compound-uniswap
https://api.coingecko.com/api/v3/coins/compound-uniswap/history?date=01-04-2018localization=False
Processing golem
URL: https://api.coingecko.com/api/v3/coins/golem/history?date=01-04-2018localization=False
At index: 150
Processing axie-infinity
URL: https://api.coingecko.com/api/v3/coins/axie-infinity/history?date=01-04-2018localization=False
At index: 1

Processing steem
URL: https://api.coingecko.com/api/v3/coins/steem/history?date=01-04-2018localization=False
At index: 175
Processing true-usd
URL: https://api.coingecko.com/api/v3/coins/true-usd/history?date=01-04-2018localization=False
At index: 176
Processing wax
URL: https://api.coingecko.com/api/v3/coins/wax/history?date=01-04-2018localization=False
At index: 177
Processing fetch-ai
URL: https://api.coingecko.com/api/v3/coins/fetch-ai/history?date=01-04-2018localization=False
At index: 178
Key error. Coin too young to find all data
fetch-ai
https://api.coingecko.com/api/v3/coins/fetch-ai/history?date=01-04-2018localization=False
Processing akash-network
URL: https://api.coingecko.com/api/v3/coins/akash-network/history?date=01-04-2018localization=False
At index: 179
Key error. Coin too young to find all data
akash-network
https://api.coingecko.com/api/v3/coins/akash-network/history?date=01-04-2018localization=False
Processing haven
URL: https://api.coingecko.com/api/v3/coins/haven/

...

Key error. Coin too young to find all data
idena
https://api.coingecko.com/api/v3/coins/idena/history?date=01-04-2018localization=False
Processing total-crypto-market-cap-token
URL: https://api.coingecko.com/api/v3/coins/total-crypto-market-cap-token/history?date=01-04-2018localization=False
At index: 1082
Key error. Coin too young to find all data
total-crypto-market-cap-token
https://api.coingecko.com/api/v3/coins/total-crypto-market-cap-token/history?date=01-04-2018localization=False
Processing bismuth
URL: https://api.coingecko.com/api/v3/coins/bismuth/history?date=01-04-2018localization=False
At index: 1083
Processing counterparty
URL: https://api.coingecko.com/api/v3/coins/counterparty/history?date=01-04-2018localization=False
At index: 1084
Processing bittube
URL: https://api.coingecko.com/api/v3/coins/bittube/history?date=01-04-2018localization=False
At index: 1085
Processing essentia
URL: https://api.coingecko.com/api/v3/coins/essentia/history?date=01-04-2018localization=False

Key error. Coin too young to find all data
power-index-pool-token
https://api.coingecko.com/api/v3/coins/power-index-pool-token/history?date=01-04-2018localization=False
Processing trabzonspor-fan-token
URL: https://api.coingecko.com/api/v3/coins/trabzonspor-fan-token/history?date=01-04-2018localization=False
At index: 1115
Key error. Coin too young to find all data
trabzonspor-fan-token
https://api.coingecko.com/api/v3/coins/trabzonspor-fan-token/history?date=01-04-2018localization=False
Ruh Roh! Looks like you have been timed out
The last coin to be pulled was trabzonspor-fan-token at index 1115
Counting... 1
Counting... 2
Counting... 3
Counting... 4
Counting... 5
Counting... 6
Counting... 7
Counting... 8
Counting... 9
Counting... 10
Counting... 11
Counting... 12
Counting... 13
Counting... 14
Counting... 15
Counting... 16
Counting... 17
Counting... 18
Counting... 19
Counting... 20
Counting... 21
Counting... 22
Counting... 23
Counting... 24
Counting... 25
Counting... 26
Counting... 27

Key error. Coin too young to find all data
swag-finance
https://api.coingecko.com/api/v3/coins/swag-finance/history?date=01-04-2018localization=False
Processing xsigma
URL: https://api.coingecko.com/api/v3/coins/xsigma/history?date=01-04-2018localization=False
At index: 1135
Key error. Coin too young to find all data
xsigma
https://api.coingecko.com/api/v3/coins/xsigma/history?date=01-04-2018localization=False
Processing iteration-syndicate
URL: https://api.coingecko.com/api/v3/coins/iteration-syndicate/history?date=01-04-2018localization=False
At index: 1136
Key error. Coin too young to find all data
iteration-syndicate
https://api.coingecko.com/api/v3/coins/iteration-syndicate/history?date=01-04-2018localization=False
Processing litecoin-cash
URL: https://api.coingecko.com/api/v3/coins/litecoin-cash/history?date=01-04-2018localization=False
At index: 1137
Processing cryptokek
URL: https://api.coingecko.com/api/v3/coins/cryptokek/history?date=01-04-2018localization=False
At index: 113

Key error. Coin too young to find all data
team-heretics-fan-token
https://api.coingecko.com/api/v3/coins/team-heretics-fan-token/history?date=01-04-2018localization=False
Processing midas
URL: https://api.coingecko.com/api/v3/coins/midas/history?date=01-04-2018localization=False
At index: 1168
Key error. Coin too young to find all data
midas
https://api.coingecko.com/api/v3/coins/midas/history?date=01-04-2018localization=False
Processing tokenomy
URL: https://api.coingecko.com/api/v3/coins/tokenomy/history?date=01-04-2018localization=False
At index: 1169
Processing gridcoin-research
URL: https://api.coingecko.com/api/v3/coins/gridcoin-research/history?date=01-04-2018localization=False
At index: 1170
Processing hyve
URL: https://api.coingecko.com/api/v3/coins/hyve/history?date=01-04-2018localization=False
At index: 1171
Key error. Coin too young to find all data
hyve
https://api.coingecko.com/api/v3/coins/hyve/history?date=01-04-2018localization=False
Processing utu-coin
URL: https://a

KeyboardInterrupt: 

In [18]:
len(coin_data_2018)

1175

In [21]:
coins_2018_df = pd.DataFrame(coin_data_2018)
coins_2018_df.id.nunique()


1165

In [22]:
coins_2018_df.to_csv("data/social_data_coins_2018.csv")

